In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
from tqdm import tqdm
tqdm.pandas()
import ast

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pickle

In [ ]:
vectorizer = pickle.load(open("/content/drive/MyDrive/master_thesis/baselines/vectorizer/tfidf.pickle", "rb"))

In [ ]:
mask = np.load('/content/drive/MyDrive/master_thesis/baselines/tf-idf_baseline_ga/mask/best_mask.npz')['mask']

In [ ]:
def vectorize(x):
  return vectorizer.transform(x).multiply(mask)

In [ ]:
def get_case_embedding(df, case_id):
  return df[df['CaseID'] == case_id].iloc[0]['embedding']

In [ ]:
def get_similarity_with_case_descriptions(df, embedding, case_id):
  case_embedding = get_case_embedding(df, case_id)
  return cosine_similarity(case_embedding, embedding).flatten()[0]

In [ ]:
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

In [ ]:
def convert_y_to_label(y):
  labels = []
  for item in y:
    if item:
      labels.append('associated_paper')
    else:
      labels.append('irrelevant_paper')
  return labels

In [ ]:
def plot_cm(target_y, pred_y, labels):
  cm = confusion_matrix(target_y, pred_y, labels=labels)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot()
  plt.show()

# Test with mask

In [ ]:
test_papers = pd.read_csv('/content/drive/MyDrive/master_thesis/dataset_data/test/test_papers.csv')
test_cases_data = pd.read_csv('/content/drive/MyDrive/master_thesis/dataset_data/test/test_cases.csv')
test_cases_data['ChallengeDescription_stemmed'] = test_cases_data['ChallengeDescription_stemmed'].progress_apply(ast.literal_eval)
test_papers['abstract_stemmed_joined'] = test_papers['abstract_stemmed'].apply(lambda x: ' '.join(x))

In [ ]:
test_cases_data['ChallengeDescription_stemmed_joined'] = test_cases_data['ChallengeDescription_stemmed'].apply(lambda x: ' '.join(x))

In [ ]:
test_cases_data['embedding'] = test_cases_data['ChallengeDescription_stemmed_joined'].progress_apply(lambda x: vectorize([x]))

In [ ]:
test_papers['embedding'] = test_papers['abstract_stemmed_joined'].progress_apply(lambda x: vectorize([x]))

In [ ]:
test_papers['case_embedding_similarity'] = test_papers.progress_apply(lambda x: get_similarity_with_case_descriptions(test_cases_data, x.embedding, x.case_id), axis=1)

In [ ]:
optimal_threshold = -0.01031448

In [ ]:
test_papers['pred_is_associated'] = test_papers['case_embedding_similarity'].apply(lambda x: x >= optimal_threshold)

In [ ]:
y_test_target = convert_y_to_label(test_papers['type'] == 'associated_paper')
y_test_pred = convert_y_to_label(test_papers['pred_is_associated'])
plot_cm(y_test_target, y_test_pred, ['associated_paper', 'irrelevant_paper'])
print(classification_report(y_test_target, y_test_pred, digits=4))